# **Library**

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from tensorflow.keras.models import Sequential

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Dataset**

**Load Dataset**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
train_data = pd.read_csv('/content/gdrive/My Drive/purify/dataset/train.txt', header=None)
val_data = pd.read_csv('/content/gdrive/My Drive/purify/dataset/val.txt', header=None)
test_data = pd.read_csv('/content/gdrive/My Drive/purify/dataset/test.txt', header=None)

In [ ]:
train_data.head(10)

,0
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger
5,ive been feeling a little burdened lately wasn...
6,ive been taking or milligrams or times recomme...
7,i feel as confused about life as a teenager or...
8,i have been with petronas for years i feel tha...
9,i feel romantic too;love


**Prepare Dataset**

In [ ]:
def rename_column(dataset):
    dataset.rename(columns = {0: "description"}, inplace = True)
    return dataset

In [ ]:
#rename column
rename_column(train_data)
rename_column(val_data)
rename_column(test_data)

,description
0,im feeling rather rotten so im not very ambiti...
1,im updating my blog because i feel shitty;sadness
2,i never make her separate from me because i do...
3,i left with my bouquet of red and yellow tulip...
4,i was feeling a little vain when i did this on...
...,...
1995,i just keep feeling like someone is being unki...
1996,im feeling a little cranky negative after this...
1997,i feel that i am useful to my people and that ...
1998,im feeling more comfortable with derby i feel ...


In [ ]:
def prepare_table(dataset):
    dataset[["text", "emotion"]] = dataset["description"].str.split(";", expand = True)
    dataset = dataset.drop("description", axis = 1)
    return dataset

In [ ]:
#prepare table
train_data = prepare_table(train_data)
val_data = prepare_table(val_data)
test_data = prepare_table(test_data)

**Remove Stopwords**

In [ ]:
def remove_stopwords(input_text):
    words = input_text.split()
    clean_words = [word for word in words if word not in stopwords.words("english")]
    clean_words = " ".join(clean_words)
    return clean_words

In [ ]:
train_data["text"] = train_data["text"].apply(remove_stopwords)
val_data["text"] = val_data["text"].apply(remove_stopwords)
test_data["text"] = test_data["text"].apply(remove_stopwords)

In [ ]:
train_data.head(10)

,text,emotion
0,didnt feel humiliated,sadness
1,go feeling hopeless damned hopeful around some...,sadness
2,im grabbing minute post feel greedy wrong,anger
3,ever feeling nostalgic fireplace know still pr...,love
4,feeling grouchy,anger
5,ive feeling little burdened lately wasnt sure,sadness
6,ive taking milligrams times recommended amount...,surprise
7,feel confused life teenager jaded year old man,fear
8,petronas years feel petronas performed well ma...,joy
9,feel romantic,love


# **Pre-processing**

In [ ]:
def preprocess(dataset):
    score = []
    for i in range(len(dataset.emotion)):
        # happy emotion : joy, love, surprise
        if dataset.emotion[i] in ['joy', 'love', 'surprise']:
            score.append(1) # happy is 1
        #sad emotion : anger, fear, sadness
        else:
            score.append(0) # sad is 0
    dataset['score'] = score
    return dataset

In [ ]:
train_data = preprocess(train_data)
test_data = preprocess(test_data)
val_data = preprocess(val_data)

# **Model**

**Transfer learning**

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
Total params: 48,190,600
Trainable params: 48,190,600
Non-trainable params: 0
_________________________________________________________________


**Building model**

In [ ]:
new_model = tf.keras.models.Sequential([hub_layer,
                                    tf.keras.layers.Dense(256, activation= 'relu'),
                                    tf.keras.layers.Dense(64, activation = 'relu'),
                                    tf.keras.layers.Dense(1, activation= 'sigmoid')])

In [ ]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_3 (Dense)             (None, 256)               13056     
                                                                 
 dense_4 (Dense)             (None, 64)                16448     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 48,220,169
Trainable params: 48,220,169
Non-trainable params: 0
_________________________________________________________________


In [ ]:
new_model.compile(optimizer = tf.optimizers.Adam(learning_rate= 0.0001),
                loss = tf.losses.BinaryCrossentropy(),
                metrics = [tf.metrics.BinaryAccuracy(name = 'accuracy')])

In [ ]:
history = new_model.fit(train_data.text, train_data.score,
                      epochs = 20,
                      batch_size = 256,
                      validation_data = (val_data.text, val_data.score))

Epoch 1/20
63/63 [==============================] - 35s 534ms/step - loss: 0.6785 - accuracy: 0.5933 - val_loss: 0.6643 - val_accuracy: 0.6095
Epoch 2/20
63/63 [==============================] - 35s 556ms/step - loss: 0.6342 - accuracy: 0.7017 - val_loss: 0.6086 - val_accuracy: 0.7445
Epoch 3/20
63/63 [==============================] - 34s 544ms/step - loss: 0.5620 - accuracy: 0.7746 - val_loss: 0.5282 - val_accuracy: 0.7970
Epoch 4/20
63/63 [==============================] - 33s 531ms/step - loss: 0.4711 - accuracy: 0.8106 - val_loss: 0.4470 - val_accuracy: 0.8245
Epoch 5/20
63/63 [==============================] - 34s 535ms/step - loss: 0.3909 - accuracy: 0.8422 - val_loss: 0.3964 - val_accuracy: 0.8465
Epoch 6/20
63/63 [==============================] - 33s 531ms/step - loss: 0.3289 - accuracy: 0.8706 - val_loss: 0.3479 - val_accuracy: 0.8615
Epoch 7/20
63/63 [==============================] - 34s 545ms/step - loss: 0.2776 - accuracy: 0.8922 - val_loss: 0.3150 - val_accuracy: 0.8765

# **Evaluate**

In [ ]:
test = preprocess(test_data)
results = new_model.evaluate(test['text'], test['score'])

print(results)

63/63 [==============================] - 5s 79ms/step - loss: 0.1982 - accuracy: 0.9335
[0.19823013246059418, 0.9334999918937683]


# **Prediction and Result**

In [ ]:
prediction = new_model.predict(train_data.text)

In [ ]:
def postprocessing(preds):
  range = prediction.max()-prediction.min()
  norm_preds = []
  probab = []
  for i in preds:
    norm_preds.append((i - prediction.min()) / range)
    probab.append((i - prediction.min()) * 100 / range)
  return np.mean(probab)

# **Checking prediction**


Positive sentiment

In [ ]:
answers = []
#Q1
print('Can you describe your life in one word?')
answers.append(input())

#Q2
print('How do you describe your feeling now ?')
answers.append(input())

#Q3
print('How would you describe your experience at your workplace/college/school in the past few days?')
answers.append(input())

#Q4
print('How do you like to spend your leisure time? How do you feel after it?')
answers.append(input())

#Q5
print('Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?')
answers.append(input())

results = new_model.predict(answers)
print('Your mental health score is:', postprocessing(results))

score = postprocessing(results)
if score < 25:
    print("You are going through a bad phase in life. But don't worry, bad times are not permanent. Try to seek help from a trained professional to improve your mental health.")
else:
    print("Your mental health looks great! Continue enjoying life and try to help others who are struggling with their mental health.")

Can you describe your life in one word?
Great
How do you describe your feeling now ?
I am feeling really carefree now
How would you describe your experience at your workplace/college/school in the past few days?
It is run smootly
How do you like to spend your leisure time? How do you feel after it?
I like to have me time, just sleeping and listeing a song or watch a movie. After that i feel fresh
Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?
I will dissapear for few times to manage my emotions
Your mental health score is: 78.01993
Your mental health looks great! Continue enjoying life and try to help others who are struggling with their mental health.


Negative Sentimen

In [ ]:
answers = []
#Q1
print('Can you describe your life in one word?')
answers.append(input())

#Q2
print('How do you describe your feeling now ?')
answers.append(input())

#Q3
print('How would you describe your experience at your workplace/college/school in the past few days?')
answers.append(input())

#Q4
print('How do you like to spend your leisure time? How do you feel after it?')
answers.append(input())

#Q5
print('Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?')
answers.append(input())

results = new_model.predict(answers)
print('Your mental health score is:', postprocessing(results))

score = postprocessing(results)
if score < 25:
    print("You are going through a bad phase in life. But don't worry, bad times are not permanent. Try to seek help from a trained professional to improve your mental health.")
else:
    print("Your mental health looks great! Continue enjoying life and try to help others who are struggling with their mental health.")

Can you describe your life in one word?
Bad
How do you describe your feeling now ?
I am feeling frustated with my life this whole week
How would you describe your experience at your workplace/college/school in the past few days?
i feel selfish and spoiled
How do you like to spend your leisure time? How do you feel after it?
i feel pretty pathetic most of the time
Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?
i suppose he feels badly because he was a bit skeptical of her pain over the last few months shes had a hyperchondria and exaggeration habit in the past though he never openly questioned her about it
Your mental health score is: 20.394438
You are going through a bad phase in life. But don't worry, bad times are not permanent. Try to seek help from a trained professional to improve your mental health.


# **Save and convert model**

In [ ]:
new_model.save('purify_v1.h5')

In [ ]:
# Converting a tf.Keras model to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmprtfcbrv9/assets


INFO:tensorflow:Assets written to: /tmp/tmprtfcbrv9/assets


In [ ]:
TF_LITE_MODEL_FILE_NAME = "purify_v1(2).tflite"
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

192887988